# Integration Tests for Deno Jupyter
This notebook contains a number of tests to ensure that Jupyter is working as expected. You should be able to select "Kernel -> Restart Kernel and Run All" in Jupyter's notebook UI to run the tests.

## Passing Tests

### Simple Tests

#### This test should print "hi".
If this doesn't work, everything else will probably fail :)

In [1]:
console.log("hi")

hi


#### Top-level await

In [2]:
let x = await Promise.resolve(42);
console.log("x is", x);

x is 42


#### TypeScript transpiling
Credit to [typescriptlang.org](https://www.typescriptlang.org/docs/handbook/interfaces.html) for this code

In [4]:
interface SquareConfig {
  color?: string;
  width?: number;
}
 
function createSquare(config: SquareConfig): { color: string; area: number } {
  return {
    color: config.color || "red",
    area: config.width ? config.width * config.width : 20,
  };
}
 
createSquare({ colour: "red", width: 100 });

{ color: "red", area: 10000 }

### Return Values

#### undefined should not return a value

In [5]:
undefined

#### null should return "null"

In [6]:
null

null

#### boolean should return the boolean

In [7]:
true

true

#### number should return the number

In [8]:
42

42

#### string should return the string

In [9]:
"this is a test of the emergency broadcast system"

"this is a test of the emergency broadcast system"

#### bigint should return the bigint in literal format

In [10]:
31337n

31337n

#### symbol should return a string describing the symbol

In [11]:
Symbol("foo")

Symbol(foo)

#### object should describe the object inspection

In [12]:
{foo: "bar"}

{ foo: "bar" }

#### resolve returned promise

In [13]:
Promise.resolve("it worked!")

Promise { "it worked!" }

In [14]:
Promise.reject(new Error("it failed!"));

Promise {
  <rejected> Error: it failed!
    at <anonymous>:2:16
}

In [15]:
(function foo() {
    throw new Error("this is a test")
})()

Error: this is a test

In [16]:
throw "a party"

Unknown error: a party

In [19]:
Deno.readFile(1)

Promise {
  <rejected> NotFound: No such file or directory (os error 2): readfile ''
    at async Object.readFile (ext:deno_fs/30_fs.js:716:18) {
    name: "NotFound",
    code: "ENOENT"
  }
}

In [20]:
console.log("Hello from Deno!")

Hello from Deno!


In [21]:
console.log("%c Hello Deno ", "background-color: #15803d; color: white;");

 Hello Deno 


In [22]:
"Cool 🫡"

"Cool 🫡"

In [23]:
console.table([1, 2, 3])

┌───────┬────────┐
│ (idx) │ Values │
├───────┼────────┤
│     0 │      1 │
│     1 │      2 │
│     2 │      3 │
└───────┴────────┘


## Unit Tests With `Deno.test()`

In [1]:
Deno.test("passing test", () => {});

Deno.test("passing test with steps", async (t) => {
  await t.step("step 1", () => {});
  await t.step("step 2", () => {});
});

Deno.test("failing test", () => {
  throw new Error("some message");
});


passing test ... ok (1ms)
passing test with steps ...
  step 1 ... ok (0ms)
  step 2 ... ok (0ms)
passing test with steps ... ok (1ms)
failing test ... FAILED (1ms)

 ERRORS 

failing test => <anonymous>:7:6
error: Error: some message
    at <anonymous>:8:9

 FAILURES 

failing test => <anonymous>:7:6

FAILED | 2 passed (2 steps) | 1 failed (0ms)


## Broadcasting Display Updates

In [24]:
await Deno.jupyter.broadcast("display_data", {
    data: { "text/html": "<b>Processing.</b>" },
    metadata: {},
    transient: { display_id: "progress" }
});

await new Promise((resolve) => setTimeout(resolve, 500));

await Deno.jupyter.broadcast("update_display_data", {
    data: { "text/html": "<b>Processing..</b>" },
    metadata: {},
    transient: { display_id: "progress" }
});

await new Promise((resolve) => setTimeout(resolve, 500));

await Deno.jupyter.broadcast("update_display_data", {
    data: { "text/html": "<b>Processing...</b>" },
    metadata: {},
    transient: { display_id: "progress" }
});

await new Promise((resolve) => setTimeout(resolve, 500));

await Deno.jupyter.broadcast("update_display_data", {
    data: { "text/html": "<b>Complete ✅</b>" },
    metadata: {},
    transient: { display_id: "progress" }
});

Complete ✅

## Comms

In [25]:
// Smoke test: Send example Jupyter Widgets messages with "extra" context.
// No return because we don't have a front-end widget to get the message from.
await Deno.jupyter.broadcast(
    "comm_open",
    {
        "comm_id": "foo",
        "target_name": "jupyter.widget",
        "data": {
            "state": {},
        },
    },
    {
        "metadata": { "version": "2.1.0" },
    },
);

await Deno.jupyter.broadcast(
    "comm_msg",
    {
        "comm_id": "foo",
        "data": {
            "method": "update",
            "state": { "answer": null },
            "buffer_paths": [["answer"]]
        },
    },
    {
        "buffers": [new Uint8Array([42])],
    },
);

## Rich Classes

In [26]:
class SuperColor {
  constructor() {
      this.color = "#239814"
  }
  hex() {
      return this.color
  }
    
  [Symbol.for("Jupyter.display")]() {
      return {
          "text/html": `<div style="width: 32px; height: 32px; background-color: ${this.hex()}" />`
      }
  }
}

let sc = new SuperColor()
sc

In [27]:
class SuperColorAsync {
  constructor() {
      this.color = "#239814"
  }
  hex() {
      return this.color
  }
    
  async [Symbol.for("Jupyter.display")]() {
      return {
          "text/html": `<div style="width: 32px; height: 32px; background-color: ${this.hex()}" />`
      }
  }
}

let sc = new SuperColorAsync()
sc